In [20]:
import re
import requests
# 根据正则表达式获取北京市地铁线路 各站点连接关系
def station_graph():
    # headers可以通过浏览器打开开发者模式获取
    headers = {"User-Agent" : "User-Agent:Mozilla/5.0 (compatible; MSIE 9.0; Windows NT 6.1; "}
    url = 'https://baike.baidu.com/item/%E5%8C%97%E4%BA%AC%E5%9C%B0%E9%93%81/408485'
    pageContent = requests.get(url, headers=headers).content.decode('utf8')
    graphContent = re.findall("(table log-set-param=\"table_view\" width=\"658\".*)",pageContent)[0]
    roadLinks = re.findall("href=\"([^\"]*)\">北京地铁",graphContent)
    from collections import defaultdict
    station_connection = defaultdict(list)
    for eachlink in roadLinks:
        url ='https://baike.baidu.com'+eachlink
        text = requests.get(url, headers=headers).content.decode('utf8')
        graphlink = re.findall("(号线相邻站间距信息统计表</caption>.*)", text)
        path  = re.findall(">(\w+——\w+)<",str(graphlink))
        #dis = re.findall("\>(\d{3,4})", str(graphlink))
        for adj in path:
            node1,node2 = adj.split("——")
            station_connection[node1].append(node2)
            station_connection[node2].append(node1)
    return station_connection
station_graph = station_graph()
station_graph

defaultdict(list,
            {'七里庄': ['六里桥', '丰台东大街'],
             '万寿路': ['五棵松', '公主坟'],
             '三元桥': ['太阳宫', '亮马桥'],
             '上地': ['五道口', '西二旗'],
             '东单': ['王府井', '建国门', '灯市口', '崇文门'],
             '东四': ['张自忠路', '灯市口', '南锣鼓巷', '朝阳门'],
             '东四十条': ['朝阳门', '东直门'],
             '东夏园': ['郝家府', '潞城'],
             '东大桥': ['朝阳门', '呼家楼'],
             '东直门': ['东四十条', '雍和宫', '柳芳'],
             '中关村': ['北京大学东门', '海淀黄庄'],
             '丰台东大街': ['七里庄', '丰台南路'],
             '丰台南路': ['丰台东大街', '科怡路'],
             '丰台科技园': ['科怡路', '郭公庄'],
             '丰台站': ['首经贸', '泥洼'],
             '九龙山': ['双井', '大郊亭'],
             '五棵松': ['玉泉路', '万寿路'],
             '五道口': ['知春路', '上地'],
             '亮马桥': ['三元桥', '农业展览馆'],
             '人民大学': ['海淀黄庄', '魏公村'],
             '俸伯': ['顺义'],
             '健德门': ['牡丹园', '北土城'],
             '光熙门': ['芍药居', '柳芳'],
             '八宝山': ['八角游乐园', '玉泉路'],
             '八角游乐园': ['古城', '八宝山'],
             '公主坟': ['万寿路', '军事博物馆', '莲花

In [21]:
def get_succsssors(froninter, graph):
    return graph[froninter]
def sort_pathes(pathes, func):
    return sorted(pathes, key=func)[:5]
def search_destination(graph, start, get_succsssors,destination ):
    pathes = [[start]]
    seen = set()
    chosen_pathes = []
    while pathes:
        path = pathes.pop(0)
        frontier = path[-1]
        if frontier in seen: continue
            
        for city in get_succsssors(frontier, graph):
            if city in path: continue # remove the loop
            new_path = path + [city]
            pathes.append(new_path)
            if city == destination: return new_path
        #path = sort_pathes(pathes,lambda p : get_distance(p,destination)+get_distance(p,start))
        #path = sort_pathes(pathes,lambda p : get_distance(p,destination))
        path = sort_pathes(pathes,lambda p :len(p))
        #pathes = strategy_func(pathes)
        seen.add(frontier)
    return chosen_pathes

In [23]:
search_destination(city_graph(),'西二旗', get_succsssors,'苹果园')

['西二旗',
 '上地',
 '五道口',
 '知春路',
 '大钟寺',
 '西直门',
 '车公庄',
 '车公庄西',
 '白石桥南',
 '白堆子',
 '军事博物馆',
 '公主坟',
 '万寿路',
 '五棵松',
 '玉泉路',
 '八宝山',
 '八角游乐园',
 '古城',
 '苹果园']